<h1 align="center"><b>CS420.M11.KHCL - Selected Topics in Computer Vision</b></h1>
<h1 align="center"><b>Bài tập 2 - Xu hướng</b></h1>

**Sinh viên thực hiện:**
- Thái Trần Khánh Nguyên - 19520188

In [117]:
import requests
from bs4 import BeautifulSoup as Soup
import nltk
from nltk.corpus import stopwords

In [118]:
#Download từ điẻn stopwords
nltk.download('stopwords')
l = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [119]:
# Dùng để gửi request đến trang web với tham số là số thứ tự trang cần thu thập
def sendRequest(day, year):
  """
  This function gets html of website need to crawl title's paper

  Argument: 
    day: date of the conference CVPR - ngày diễn ra hội nghị
    year: year and month of the conference CVPR - năm và tháng diễn ra hội nghị

  Return: 
    title_tag: the entire tag contains information about the titles of the published papers.
  """

  #Tạo đường liên kết đến trang cần thu thập dữ liệu
  url = "https://openaccess.thecvf.com/CVPR" + year + day
  #Gửi request đến trang đó
  request = requests.get(url)
  #BeautifulSoup dùng để phân tích dữ liệu html thành dữ liệu cây để chúng ta dễ dàng thao tác sau này
  title_tag = Soup(request.text, 'html.parser')
  return title_tag.find_all('dt')

In [120]:
def getTitle(keywords, titles):
  """
    This function is used to separate and count words in the titles of articles

    Argument:
      keywords: dict contains the number of words that appear
      titles:   tag containing the title of the article to be split and counted

    Return:
      keywords: dict has updated the word count
  """

  common_word = ['image', 'images', 'learning', 'using', 'object', 'via', 
               'video', 'images', 'visual', 'based', 'use', 'dataset', 
               'photo', 'data', 'prediction', 'model']

  for title in titles:
    #Tách các từ và loại bỏ stopwords + các từ phổ biến trong CV của tiêu đề
    tmp = title.text.lower()
    str = [word for word in tmp.split(' ') if not (word in stopwords.words('english') or word in common_word)]
    #Cập nhật số lượng keyword
    for i in str:
      if i in keywords:
        keywords[i]+=1
      else:
        keywords[i]=1
  return keywords

In [121]:
def GetRankKeyword(count):
  """
    Ranking keywords
    -> Keywords with the same number of occurrences will rank equally

    Argument:
      count: dict contains the number of keywords that appear

    Return:
      rank: dict contains keywords's rank
  """

  #Khởi tạo dict rank để lưu rank của các keyword và index để xét rank của chúng
  rank, index = {}, 0
  #max_idx dùng để kiểm tra các keywords có thứ hạng bằng nhau
  max_idx = 1e9
  #Sắp xếp dict count và thao tác trực tiếp trên dict đã được sắp xếp
  for i in sorted(count, key=count.get, reverse=True):
    #Nếu số lần xuất hiện của keyword < max_idx thì tăng thứ hạng lên và cập nhật lại max_idx
    if (count[i] < max_idx):
      index += 1
      max_idx = count[i]
    #lưu rank của keyword
    rank[i] = index
    
  return rank


In [132]:
def trendKeyword(rank_1, rank_2):
  """
    Collect trends of CV through two years
    -> Check the ranking difference of keywords

    Argument:
      rank_1: ranking of keywords in the previous year
      rank_2: ranking of keywords in the following year

    Return:
      trend, new_trend: trend of keywords (Ranking difference between two years)
  """
  #Khởi tạo các dict rỗng, trend để lưu xu hướng qua 2 năm, new_trend các từ mới xuất hiện trong năm nay
  trend, new_trend = {}, {}
  #Xét các từ xuất hiện trong năm sau
  for word in rank_2:
    #Nếu nó có tồn tại trong năm trước thì check độ chệnh lệnh
    if (word in rank_1):
      trend[word] = rank_1[word] - rank_2[word]
    #Nếu không thì nó là từ mới cập nhật vào new_trend
    else:
      new_trend[word] = rank_2[word]
  
  return (trend, new_trend)

In [129]:
def draw_plot(trend, new_trend):
  from bokeh.plotting import figure, output_notebook, show
  X, Y = [], []
  for i in sorted(trend, key=trend.get, reverse=True):
    X.append(i)
    Y.append(trend[i])
 
  output_notebook()

  p = figure(x_range=X[:20], height=500, width = 800, title="Trend 2020-2021")
  p.vbar(x=X[:20], top=Y[:20], width=0.5,  color="coral")
  p.xaxis.major_label_orientation = 1
  p.xgrid.grid_line_color = None
  p.y_range.start = 0

  show(p)
  X, Y = [], []
  for i in sorted(new_trend, key=new_trend.get, reverse=True):
    X.append(i)
    Y.append(new_trend[i])
 
  output_notebook()

  p1 = figure(x_range=X[:20], height=500, width = 800, title="New Trend 2021")
  p1.vbar(x=X[:20], top=Y[:20], width=0.5,  color="plum")
  p1.xaxis.major_label_orientation = 1
  p1.xgrid.grid_line_color = None
  p1.y_range.start = 0
  show(p1)
  

In [130]:
def main():
  keywords_2020, keywords_2021 = {}, {}

  # Crawl keywords trong năm 2020
  for day in ['16', '17', '18']:
    titles = sendRequest(day, '2020?day=2020-06-')
    keywords_2020 = getTitle(keywords_2020, titles)

  for day in range(21, 26):
    keywords_2021 = getTitle(keywords_2021, sendRequest(str(day), '2021?day=2021-06-'))

  rank_2020, rank_2021 = GetRankKeyword(keywords_2020), GetRankKeyword(keywords_2021)
  trend, new_trend = trendKeyword(rank_2020, rank_2021)
  draw_plot(trend, new_trend)

In [133]:
if __name__ == '__main__':
    main()